In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# import os
# import numpy as np
# np.sort(os.listdir("/home/quetalasj/datasets/orb-cls/dataset_1_cam_1/images"))[:100]

In [3]:
PYTHON_PATHS = [".."]
import sys
for path in PYTHON_PATHS:
    if path not in sys.path:
        sys.path.append(path)

import torch
import torch.nn as nn
from classifier.resnet18 import Resnet18Model
from data_loader.orb_features_data_module import ORBFeaturesDataModule
import pytorch_lightning as pl

/home/nuke/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# ResNet backbone model

In [4]:
model = Resnet18Model(512, [512], 2, freeze_backbone=False)

network head
Sequential(
  (0): Linear(in_features=512, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=512, bias=True)
  (3): ReLU()
  (4): Linear(in_features=512, out_features=2, bias=True)
)


Using cache found in /home/nuke/.cache/torch/hub/pytorch_vision_v0.10.0


In [5]:
dataset_params = {'root': "/home/nuke/datasets/orb-cls/", 
                  'dataset': [
                      "dataset_1_cam_1",
                      "dataset_1_cam_2",
                      "dataset_1_cam_3",
                      "dataset_1_cam_4",
                      "dataset_1_cam_5",
                      
#                       "dataset_2_cam_0",
#                       "dataset_2_cam_1",
#                       "dataset_2_cam_2",
#                       "dataset_2_cam_3",
#                       "dataset_2_cam_4",
#                       "dataset_2_cam_5",
                             ],
                  'classification_threshold': 300,
                  'transform': model.transform,
                  'batch_size': 64,
                  'shuffle': True,
                  'num_workers': 6,
                  'num_val': 1000,
                  "num_test": 1000,
                  }

datamodule = ORBFeaturesDataModule(**dataset_params)

In [6]:
trainer = pl.Trainer(gpus=1, log_every_n_steps=1, max_epochs=2)
trainer.fit(model, datamodule)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


datasetname: dataset_1_cam_1
num images: 3824
num labels: 3824
datasetname: dataset_1_cam_2
num images: 3824
num labels: 3824
datasetname: dataset_1_cam_3
num images: 3825
num labels: 3825
datasetname: dataset_1_cam_4
num images: 3826
num labels: 3826
datasetname: dataset_1_cam_5
num images: 3825
num labels: 3825


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type       | Params
------------------------------------------
0 | activation | LogSoftmax | 0     
1 | loss       | NLLLoss    | 0     
2 | backbone   | ResNet     | 11.2 M
3 | fc         | Sequential | 526 K 
------------------------------------------
11.7 M    Trainable params
0         Non-trainable params
11.7 M    Total params
46.811    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/nuke/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:495: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(


Epoch 0:  94%|█████████▍| 268/284 [01:14<00:04,  3.61it/s, loss=0.0787, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 1:  94%|█████████▍| 268/284 [01:15<00:04,  3.55it/s, loss=0.0753, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 1: 100%|██████████| 284/284 [01:19<00:00,  3.58it/s, loss=0.0753, v_num=2]


# HardNet backbone model

In [9]:
trainer.save_checkpoint("example.ckpt")

In [10]:
!ls

evgeny	      experiment1.ipynb  lightning_logs  train_model.ipynb
example.ckpt  experiment2.ipynb  patterns.ipynb


In [11]:
model2 = Resnet18Model.load_from_checkpoint("example.ckpt", 
                                           in_size=512, hidden_sizes=[512], out_size=2)

network head
Sequential(
  (0): Linear(in_features=512, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=512, bias=True)
  (3): ReLU()
  (4): Linear(in_features=512, out_features=2, bias=True)
)


Using cache found in /home/nuke/.cache/torch/hub/pytorch_vision_v0.10.0


In [12]:
model2

Resnet18Model(
  (activation): LogSoftmax(dim=1)
  (loss): NLLLoss()
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): Ba

In [7]:
model2 = Resnet18Model.load_from_checkpoint("lightning_logs/version_3/checkpoints/epoch=1-step=507.ckpt", 
                                           in_size=512, hidden_sizes=[512], out_size=2)

FileNotFoundError: [Errno 2] No such file or directory: '/home/nuke/orb-classification/notebooks/lightning_logs/version_3/checkpoints/epoch=1-step=507.ckpt'

In [13]:
model.

AttributeError: 'Resnet18Model' object has no attribute 'weights'

In [11]:
model2

Resnet18Model(
  (activation): LogSoftmax(dim=1)
  (loss): NLLLoss()
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): Ba